# JSON Schema generation

A simple usage of the library that, given generates a JSON Schema for inputs and outputs.

## 1. Parsing

In this sample we'll show the access from a remote public URL.

In [4]:
from cwl2ogc import load_converter_from_location

cwl_converter = load_converter_from_location('https://raw.githubusercontent.com/eoap/cwl2ogc/refs/heads/develop/tests/artifacts/cwl-types/array-inputs.cwl')

# 2. Inputs JSON Schema generation

Once the document is parsed, invoke the `cwl2ogc` APIs to convert the CWL inputs to the JSON schema:

In [5]:
import sys

cwl_converter.dump_inputs_json_schema(stream=sys.stdout, pretty_print=True)

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "$id": "https://eoap.github.io/cwl2ogc/array-inputs/inputs.yaml",
  "description": "The schema to represent a array-inputs inputs definition",
  "type": "object",
  "required": [
    "filesA",
    "filesB",
    "filesC"
  ],
  "properties": {
    "filesA": {
      "$ref": "#/$defs/filesA"
    },
    "filesB": {
      "$ref": "#/$defs/filesB"
    },
    "filesC": {
      "$ref": "#/$defs/filesC"
    }
  },
  "$defs": {
    "filesA": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "filesB": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "filesC": {
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  },
  "additionalProperties": true
}

# 3. Outputs JSON Schema generation

Users can reuse the `BaseCWLtypes2OGCConverter` instance to convert the CWL outputs to the JSON Schema:

In [ ]:
cwl_converter.dump_outputs_json_schema(stream=sys.stdout, pretty_print=True)

TypeError: BaseCWLtypes2OGCConverter.get_outputs_json_schema() got an unexpected keyword argument 'stream'